# Langchain crash course

In [1]:
import os
from pprint import pprint
import json

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Taj Mahal Spice House" 


In [4]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n"Spice Delight"'

## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

{
    "cuisine": "Mexican",
    "text": "\n\"Cantina del Sol\""
}


In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
{
    "cuisine": "Mexican",
    "text": "\n\n\"El Sabor Del Sol\" (The Flavor of the Sun)"
}


In [8]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [9]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

results = chain.invoke("Indian")
print(results["input"])
pprint(results["output"])

Indian
('1. Chicken Tikka Masala: Tender pieces of chicken cooked in a creamy '
 'tomato-based sauce, served with basmati rice.\n'
 '\n'
 '2. Vegetable Biryani: Fragrant basmati rice cooked with an assortment of '
 'vegetables, nuts, and spices.\n'
 '\n'
 '3. Tandoori Shrimp: Succulent shrimp marinated in yogurt and spices, cooked '
 'in a traditional clay oven.\n'
 '\n'
 '4. Palak Paneer: Creamy spinach and cottage cheese dish, served with naan '
 'bread.\n'
 '\n'
 '5. Lamb Rogan Josh: Slow-cooked lamb in a rich and flavorful tomato-based '
 'sauce, served with rice or naan.\n'
 '\n'
 '6. Aloo Gobi: Cauliflower and potato curry with a blend of aromatic spices.\n'
 '\n'
 '7. Samosas: Crispy pastry filled with spiced potatoes and peas, served with '
 'chutney for dipping.\n'
 '\n'
 '8. Malai Kofta: Vegetable and cheese dumplings in a creamy cashew and tomato '
 'sauce.\n'
 '\n'
 '9. Butter Chicken: Tender chicken pieces in a rich and buttery tomato-based '
 'sauce, served with rice or n

#### Sequential Chain

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
results = chain.invoke({"cuisine": "Indian"})
pprint(results["cuisine"])
pprint(results["restaurant_name"])
pprint(results["menu_items"])

'Indian'
'\n\n"Taste of India"'
('\n'
 '\n'
 '1. Chicken Tikka Masala\n'
 '2. Vegetable Samosas\n'
 '3. Lamb Vindaloo\n'
 '4. Palak Paneer (spinach and cottage cheese)\n'
 '5. Tandoori Chicken\n'
 '6. Naan bread\n'
 '7. Aloo Gobi (potato and cauliflower)\n'
 '8. Chana Masala (chickpea curry)\n'
 '9. Butter Chicken\n'
 '10. Vegetable Biryani\n'
 '11. Mango Lassi (yogurt drink)\n'
 '12. Gulab Jamun (sweet dessert)\n'
 '13. Tandoori Fish\n'
 '14. Dal Makhani (lentil curry)\n'
 '15. Chicken Korma.')


## Agents

In [14]:
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [15]:
from langchain import hub
from langchain.agents import AgentType, create_react_agent, load_tools, AgentExecutor

In [16]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [17]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What was the GDP of US in 2022?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should search for the answer using a search engine.
Action: Search
Action Input: "GDP of US in 2022"$25.46 trillionI should use a calculator to convert the GDP from dollars to another currency if needed.
Action: Calculator
Action Input: $25.46 trillionAnswer: 25460000000000.0I now know the final answer.
Final Answer: The GDP of US in 2022 was $25.46 trillion.

> Finished chain.


{'input': 'What was the GDP of US in 2022?',
 'output': 'The GDP of US in 2022 was $25.46 trillion.'}

#### Wikipedia and llm-math tool

In [18]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "When was Elon musk born? What is the square root of his age as of 2023?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should use Wikipedia to find out when Elon Musk was born and then use a calculator to find the square root of his age.
Action: Wikipedia
Action Input: Elon MuskPage: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, ac

{'input': 'When was Elon musk born? What is the square root of his age as of 2023?',
 'output': "The square root of Elon Musk's age as of 2023 is approximately 7.21."}

In [19]:
7.211102550927978 ** 2

51.99999999999999

## Memory

#### ConversationBufferMemory

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabor Exquisito" (The Exquisite Flavor)'}


In [21]:
name = chain.invoke("Arabic")
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Mexican\nAI: \n\n"El Sabor Exquisito" (The Exquisite Flavor)', 'text': '\n\n"Al-Fawarish" (meaning "The Treasures" in Arabic)'}


In [22]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [23]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [24]:
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the finals, with a score of 291 runs to 274 runs. The match was held at Lord's Cricket Ground in London, England."}

In [25]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the finals, with a score of 291 runs to 274 runs. The match was held at Lord's Cricket Ground in London, England.",
 'response': ' 5+5 is equal to 10.'}

In [26]:
convo.invoke("Who was the captain ofthe winning team?")

{'input': 'Who was the captain ofthe winning team?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the finals, with a score of 291 runs to 274 runs. The match was held at Lord's Cricket Ground in London, England.\nHuman: How much is 5+5?\nAI:  5+5 is equal to 10.",
 'response': ' The captain of the West Indies team in the 1975 World Cup was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He also led the team to win the 1979 World Cup.'}

In [27]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the finals, with a score of 291 runs to 274 runs. The match was held at Lord's Cricket Ground in London, England.
Human: How much is 5+5?
AI:  5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team in the 1975 World Cup was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He also led the team to win the 1979 World Cup.


#### ConversationBufferWindowMemory

In [28]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd scoring 102 runs for his team. The winning team received a prize money of £4,000 and the runners-up received £2,000. The first ever man of the match award was given to West Indies' Vivian Richards for his outstanding performance in the final."}

In [29]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London. West Indies won by 17 runs, with Clive Lloyd scoring 102 runs for his team. The winning team received a prize money of £4,000 and the runners-up received £2,000. The first ever man of the match award was given to West Indies' Vivian Richards for his outstanding performance in the final.",
 'response': '  5+5 equals 10.'}

In [30]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: How much is 5+5?\nAI:   5+5 equals 10.',
 'response': ' I do not have enough context to answer that question. Can you provide more details about the team and the event?'}

## Streaming

In [31]:
for chunk in convo.stream("Write me a song about goldfish on the moon"):
    print(chunk["history"])
    # print(chunk.content, end="", flush=True)

Human: Who was the captain of the winning team?
AI:  I do not have enough context to answer that question. Can you provide more details about the team and the event?


In [32]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import GPT4All

# Set up the prompt template
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# Set up the local instace of the LLM
local_path = "/Users/brianroepke/Library/Application Support/nomic.ai/GPT4All/mistral-7b-openorca.Q4_0.gguf"  # noqa: E501
callbacks = [StreamingStdOutCallbackHandler()]
memory = ConversationBufferMemory()
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, streaming=True)  # noqa: E501
# llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

llm_chain = ConversationChain(llm=llm)

In [33]:
for chunk in llm_chain.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)

 Sure! Here's a little ditty I came up with:

Sparkling Water Lullaby
(To the tune of Twinkle, Twinkle Little Star)

Sparkling water so clear and bright,
Bubbles dance in the moonlight,
Quenching thirst on a summer night.

Chorus:
Drink it down, drink it up,
Feel your body tingle with delight!
Sparkling water is so right!

In the meadow where daisies play,
A cool stream flows all day,
Filling glasses for passersby.

Chorus: Drink it down...

On a mountain peak so high,
Icicles glisten in the sky,
Melting into sparkling water.

Chorus: Drink it down...

In the depths of the ocean blue,
A school of fish come swimming through,
Drinking from fountains made for two.

Chorus: Drink it down...

So next time you're parched and dry,
Remember sparkling water's why,
It quenches thirst with every sigh!
{'input': 'Write me a song about sparkling water.', 'history': '', 'response': " Sure! Here's a little ditty I came up with:\n\nSparkling Water Lullaby\n(To the tune of Twinkle, Twinkle Little Star)